# Setup

In [6]:
import torch
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
from torchvision.utils import save_image
import requests
import numpy as np

הורדת קובץ משקלי המודל המאומן מהדרופבוקס שלי

קוד להורדת קובץ נלקח מפורום באינטרנט

In [7]:
file_url = "https://www.dropbox.com/s/zk31a8d6e085e98/cool_generator.pth?dl=1"

r = requests.get(file_url, stream = True) 

with open("cool_generator.pth", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block)

קוד מודל מייצר

In [8]:
class DownsampleLayer(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, padding_mode, dropout, activation_type="relu"):
        super(DownsampleLayer, self).__init__()
        self.activation = None
        self.instancenorm = nn.InstanceNorm2d(out_channel)
        self.dropout = nn.Dropout(dropout)
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, padding_mode=padding_mode)

        if activation_type == "relu":
            self.activation = nn.ReLU(inplace=True)
        elif activation_type == "l_relu":
            self.activation = nn.LeakyReLU(0.2,inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.instancenorm(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x
#========================================================================================
class UpsampleLayer(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, out_padding, upsample_type, padding_mode):
        super(UpsampleLayer, self).__init__()
        
        if upsample_type == "conv":
            self.main = nn.Sequential(
                nn.ConvTranspose2d(in_channel, out_channel, kernel_size, stride, padding,
                                   output_padding=out_padding , padding_mode=padding_mode),
                nn.InstanceNorm2d(out_channel),
                nn.ReLU()
            )
        elif upsample_type == "conv_n_pool":
            self.main == nn.Sequential(
                nn.Conv2d(in_channel, in_channel, kernel_size, 1, padding='same', padding_mode=padding_mode),
                nn.Upsample(scale_factor=stride),
                nn.ReLU()
            )
        self.main = self.main

    def forward(self, x):
        x = self.main(x)

        return x
#========================================================================================
class ResidualBlock(nn.Module):

    def __init__(self, input_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, input_channels, kernel_size=3, padding=1, padding_mode='reflect')
        self.conv2 = nn.Conv2d(input_channels, input_channels, kernel_size=3, padding=1, padding_mode='reflect')
        self.instancenorm = nn.InstanceNorm2d(input_channels)
        self.activation = nn.ReLU()

    def forward(self, x):

        original_x = x.clone()
        x = self.conv1(x)
        x = self.instancenorm(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.instancenorm(x)
        return original_x + x
#========================================================================================
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()

        self.DSL1 = DownsampleLayer(3, 64, 7, 1, 'same', "reflect", 0)
        self.DSL2 = DownsampleLayer(64,128, 3, 2, 1, "reflect",0)
        self.DSL3 = DownsampleLayer(128,256,3,2, 1, "reflect", 0)

        self.res1 = ResidualBlock(256)
        self.res2 = ResidualBlock(256)
        self.res3 = ResidualBlock(256)
        self.res4 = ResidualBlock(256)
        self.res5 = ResidualBlock(256)
        self.res6 = ResidualBlock(256)
        self.res7 = ResidualBlock(256)
        self.res8 = ResidualBlock(256)
        self.res9 = ResidualBlock(256)

        self.USL1 = UpsampleLayer(256,128,3,2,1,1,"conv", "zeros")
        self.USL2 = UpsampleLayer(128,64,3,2,1,1,"conv", "zeros")
        self.conv1 = nn.Conv2d(64, 3, 7, 1, padding='same', padding_mode="reflect")
        self.activation = nn.Tanh()

    def forward(self, x):

        x1 = self.DSL1(x)
        x2 = self.DSL2(x1)
        x3 = self.DSL3(x2)

        x4 = self.res1(x3)
        x5 = self.res2(x4)
        x6 = self.res3(x5)
        x7 = self.res4(x6)
        x8 = self.res5(x7)
        x9 = self.res6(x8)
        x10 = self.res7(x9)
        x11 = self.res8(x10)
        x12 = self.res9(x11)

        x13 = self.USL1(x12)
        x14 = self.USL2(x13)
        x15 = self.conv1(x14)
        xn = self.activation(x15)

        return xn

יצירת מודל, טעינת המשקלים עליו, לשים על מצב של שימוש ולא אימון, מעביר לזיכרון

פונקציות לעיבוד תמונה והפיכה לטנסור

In [9]:
cool_Model = Generator()
cool_Model.load_state_dict(torch.load('/content/cool_generator.pth',map_location=torch.device('cuda')))
cool_Model.eval()
cool_Model.to('cuda')

Imagetrans = transforms.Compose([transforms.ToTensor()])
Imagetrans2 = transforms.Compose([transforms.Resize(1500), transforms.ToTensor()])

# Upload Image

In [11]:
#@title יש להריץ וללחוץ על כפתור Choose Files
maybeImage = files.upload()

try:
  for fn in maybeImage.keys():
    path = fn

  im = Image.open(path)
  im = im.convert('RGB')

  Imageimage = Imagetrans(im).unsqueeze(0).to('cuda')
  aNumber = Imageimage.shape[2]*Imageimage.shape[3]
  if aNumber>2250000:
      print('image to large')
      Imageimage = Imagetrans2(im).unsqueeze(0).to('cuda')
except:
  print("make sure to upload IMAGE, and Setup is done")

MessageError: ignored

# Generate Image

In [ ]:
#@title ייצור תמונה

try:
  with torch.no_grad():
        outputs = cool_Model.forward(Imageimage)
  torch.cuda.empty_cache()
except:
  print("make sure to upload image(more than 30x30 and less than 4000x4000), and Setup is done")

#   

In [ ]:
#@title תצוגה מקדימה של התמונה
try:
  print('generated image preview: ')
  plt.imshow((outputs.detach().cpu().squeeze().permute(1, 2, 0) + 1) / 2)
except:
  print("Generate image first")

In [ ]:
#@title הורדת התמונה
#range of num in tensor is -1 to 1, we need 0 to 1
try:
  save_image((outputs + 1) / 2, 'generated_image.png')
  files.download('generated_image.png')
except:
  print("Generate image first")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>